# Some notes on the KZ and LZ experiments, and related experiments

## Where to learn more

[Coherent quantum annealing in a
programmable 2,000 qubit Ising chain,
Nature Physics (2022)](https://doi.org/10.1038/s41567-022-01741-6) Figure 2A reproduced ([preprint](https://doi.org/10.48550/arXiv.2202.05847))

[Quantum critical dynamics in a
5,000-qubit programmable spin glass,
Nature (2023)](https://doi.org/10.1038/s41586-023-05867-2) Figure 2C reproduced ([preprint](https://doi.org/10.48550/arXiv.2207.13800))

[Other publications](https://www.dwavequantum.com/learn/publications/)

[youtube channel](https://www.youtube.com/@dwavequantum)

[Per QPU-properties](https://docs.dwavequantum.com/en/latest/quantum_research/solver_properties_specific.html) (schedules, energy scales, single-qubit freeze out 's') 

[Documentation](https://docs.dwavequantum.com/en/latest/index.html)

[github](https://github.com/dwavesystems/)

[Leap examples](https://cloud.dwavesys.com/leap/examples)

[Leap resources](https://cloud.dwavesys.com/leap/resources)

## Embedding:

Minorminer has a variety of heuristic tools, and tools with guaranteed performance:

* general mappings: [minorminer.find_embeddings](https://docs.dwavequantum.com/en/latest/ocean/api_ref_system/generated/minorminer.find_embedding.html#minorminer-find-embedding)
* 1:1 mappings (native/subgraph): **[minorminer.subgraph.find_embedding]()**
* Clique and Biclique embedding: [minorminer.busclique.find_clique_embedding](https://docs.dwavequantum.com/en/latest/ocean/api_ref_minorminer/source/clique_embedding.html#minorminer.busclique.find_clique_embedding)

Choose the right tool, it can be very inefficient to make a bad choice. Consider a customized approach for regular cases, e.g.:
* Defect-free (1:1) Next-neighbor-square, Kings, square, triangle); or Defect-free (chains) cubic [hybrid.decomposers.make_origin_embeddings](https://docs.dwavequantum.com/en/latest/ocean/api_ref_hybrid/generated/hybrid.decomposers.make_origin_embeddings.html#hybrid.decomposers.make_origin_embeddings):

Parallelization of embeddings recently supported:
**[minorminer.utils.parallel_embedding.find_multiple_embeddings]()**

In [3]:
from minorminer.subgraph import find_subgraph
from dwave.system import DWaveSampler
h = {i: 0 for i in range(3)}
J = {(0,1): 1, (1,2): 1, (0,2): 1}  # An antiferromagnetic 3-cycle
qpu = DWaveSampler()
emb1to1 = find_subgraph(J.keys(), qpu.to_networkx_graph())  # Edge list 
print(emb1to1)
emb = {k: (v,) for k, v in emb1to1.items()}

{0: 3194, 1: 3209, 2: 2760}


## Noise-mitigation/investigation:
Most of our research papers involve shimming. The general-purpose calibration can be refined for specific models to eliminate specific cross-talks and non-idealities:
* [Calibration refinement](https://doi.org/10.3389/fcomp.2023.1238988) Published paper and (recently updated) demonstration code; We can tune parameters such as flux_biases to restore expected symmetries.
* We can extend this in principle to learning general distributions [pytorch plug-in](https://github.com/dwavesystems/dwave-pytorch-plugin/blob/main/README.rst)

Another option is to use spin-reversal transforms: 
* Spin-reversal transforms with respect to the noise-free evolution amounts to a change of basis (gauge)
* With respect to the programmed realization, a change of basis leads to new noise patterns.
* By averaging we eliminate noise. e.g. the effect of independent and identically distributed flux (h) noise is eliminated at leading order.
```
    qpu = DWaveSampler(solver='Advantage2_prototype2.6')
    sampler = SpinReversalTransformComposite(qpu)  # Composites applies (and inverts) the transformation
```

Another option is averaging over embeddings:
* Use randomized or parallelized embeddings
```
    qpu = DWaveSampler(solver='Advantage2_prototype2.6')
    sampler1 = SpinReversalTransformComposite(qpu)  
    source = nx.from_edgelist([(i, i+1) for i in range(3)])  # A path (1d chain) of size 4
    sampler2 = ParallelEmbeddingComposite(sampler2) # Composites distributes and inverts (a distribution on 4-spins is returned)
```
* Exploit symmetries of the source model (Ising model) to permute a known embedding.
* Many physics models have high symmetry: tranlational, rotational and reflection symmetry. These might be put in by hand, or a tool like pynauty can be used (see e.g. the shimming tutorial).


In [4]:
from dwave.embedding import is_valid_embedding
import networkx as nx
translation_on_3cycle = {0:1, 1:2, 2:0}
emb2 = {translation_on_3cycle[k]: v for k,v in emb.items()}
print(emb)
assert is_valid_embedding(emb, J.keys(), qpu.to_networkx_graph())
print(emb2)
assert is_valid_embedding(emb2, J.keys(), qpu.to_networkx_graph())

{0: (3194,), 1: (3209,), 2: (2760,)}
{1: (3194,), 2: (3209,), 0: (2760,)}


In [7]:
from dwave.preprocessing import SpinReversalTransformComposite
from dwave.system.composites import FixedEmbeddingComposite
sampler = FixedEmbeddingComposite(SpinReversalTransformComposite(qpu), embedding=emb)
ss = sampler.sample_ising(h, J, num_reads=1000)
print(ss.record.energy, ss.record.num_occurrences)

[-1. -1. -1. -1. -1. -1.] [ 92 226 187 257 105 133]


## Temperature (quasi-static freezeout)

If the model is maximum entropy subject to some rate of excitations (energy), it is a Boltzmann distribution.

This is expected for single-late freeze-out (thermal equilibrium up to the small A(s) limit, then dynamical arrest)

The local pattern of excitations is sufficient to efficiently infer the temperature with enough samples:
```
    qpu = DWaveSampler(solver='Advantage2_prototype2.6')
    dwave.system.temperatures.fast_effective_temperature(qpu)  # By default, uses independent spins
```
Note that, if the distribution is not Boltzmann due to ergodicity breaking (trapping by local minima) we expect the rate of local excitations to be smaller than the rate of general excitations. Quantum-Boltzmann and noise effects can also lead to bias in estimators.

In [16]:
from dwave.system.temperatures import fast_effective_temperature
from dwave.system import DWaveSampler
qpu = DWaveSampler(solver='Advantage2_prototype2.6')
T1, _ = fast_effective_temperature(qpu)  # By default, uses independent spins
qpu = DWaveSampler(solver='Advantage_system4.1')
T2, _ = fast_effective_temperature(qpu)  # By default, uses independent spins
print('Unitless temperatures characterizing the two QPUs', T1, T2)

Unitless temperatures characterizing the two QPUs 0.11744768421923785 0.16884139784615673


## Classical dynamics

We include a variety of heuristic optimizer and classical dynamics include path-integral monte carlo, rotor models and simulated annealing

In [ ]:
from dwave.samplers import SimulatedAnnealingSampler
sampler = SimulatedAnnealingSampler()
sampler.sample_ising(h, J, randomize_order=True, acceptance_criteria='Gibbs', num_sweeps = 16, beta_schedule=[1/T1,1/T1])